In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [ ]:
import sklearn
print(sklearn.__version__)

1.2.2


In [ ]:
tickers = ['AAPL', 'GOOG', 'AMZN', 'NVDA', 'MSFT',"^GSPC", "^IXIC"]

df = yf.download(tickers, start='2010-01-01', end='2024-01-01')

[*********************100%%**********************]  7 of 7 completed


In [ ]:
df.dropna(inplace=True)
normalize = MinMaxScaler()
normalized_data = normalize.fit_transform(df['Close'].values.reshape(-1, 1))

train_size = int(len(normalized_data) * 0.7)
test_size = len(normalized_data) - train_size
train_data, test_data = normalized_data[0:train_size,:], normalized_data[train_size:len(normalized_data),:]

In [ ]:
def generate_dataset(original_data, look_back=1):
    data_X, data_Y = [], []
    for i in range(len(original_data)-look_back-1):
        a = original_data[i:(i+look_back), 0]
        data_X.append(a)
        data_Y.append(original_data[i + look_back, 0])
    return np.array(data_X), np.array(data_Y)

In [ ]:
look_back = 40
X_train, y_train = generate_dataset(train_data, look_back)
X_test, y_test = generate_dataset(test_data, look_back)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

model = Sequential()
model.add(LSTM(units=100, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=100))
model.add(Dropout(0.2))
model.add(Dense(units=1))

In [ ]:
model.compile(optimizer='SGD', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=80, batch_size=32)

Epoch 1/80
538/538 [==============================] - 8s 9ms/step - loss: 5.3895e-05
Epoch 2/80
538/538 [==============================] - 5s 10ms/step - loss: 5.2883e-05
Epoch 3/80
538/538 [==============================] - 4s 8ms/step - loss: 4.9421e-05
Epoch 4/80
538/538 [==============================] - 4s 8ms/step - loss: 4.9192e-05
Epoch 5/80
538/538 [==============================] - 5s 9ms/step - loss: 4.8814e-05
Epoch 6/80
538/538 [==============================] - 5s 8ms/step - loss: 4.8804e-05
Epoch 7/80
538/538 [==============================] - 5s 9ms/step - loss: 4.7987e-05
Epoch 8/80
538/538 [==============================] - 5s 8ms/step - loss: 4.8278e-05
Epoch 9/80
538/538 [==============================] - 4s 8ms/step - loss: 4.7405e-05
Epoch 10/80
538/538 [==============================] - 5s 10ms/step - loss: 4.7473e-05
Epoch 11/80
538/538 [==============================] - 5s 9ms/step - loss: 4.8187e-05
Epoch 12/80
538/538 [==============================] - 5s 9ms

In [ ]:
import tensorflow as tf
y_predict = model.predict(X_test)
error_rate = np.sqrt(mean_squared_error(y_test, y_predict))
print('Error Rate - ', error_rate)

230/230 [==============================] - 2s 5ms/step
Error Rate -  0.08951277083991442


In [ ]:
accuracy_eval = tf.keras.metrics.Accuracy()
accuracy_eval.update_state(tf.math.round(y_test), tf.math.round(y_predict))
acc = accuracy_eval.result().numpy()
print('Accuracy:', acc)

Accuracy: 0.9990471


In [ ]:
import pickle
from joblib import Parallel, delayed
import joblib
joblib.dump(model, 'lstm_model.pkl')

['lstm_model.pkl']